# SISTEMAS DE RECOMENDACIÓN

# PLANTEAMIENTO DEL PROBLEMA

La asociación Sukalde se enfrenta a la necesidad de optimizar sus operaciones en el contexto de una cocina comunitaria en donde la planificación de los menús y la satisfacción de los usuarios son dos factores muy importantes para el buen desarrollo de su labor.

Como entidad con pocos años de funcionamiento y recursos humanos limitados, se enfrenta a desafíos operativos relacionados con una planificación de menús semanales diversos y una búsqueda eficaz de la satisfacción de los usuarios. El propósito de este estudio es abordar estas problemáticas mediante la creación de sistemas de recomendación que se basen en la clasificación de los platos en familias y la similitud de los usuarios.

A través de este enfoque se permitirá la automatización y la mejora de los procesos de planificación de menús, la adquisición de los ingredientes y la personalización de las sugerencias de platos para los usuarios, aportando así un valor significativo a la asociación.

Se crearán un total de 4 sistemas de recomendación que se dividirán en dos grupos:

A) **Sistemas de recomendación para el equipo de cocina** - Los dos primeros sistemas de recomendación irán destinados al equipo de cocina de la asociación, de manera que les ayude a la creación de nuevos menús semana a semana.

B) **Sistemas de recomendación para los usuarios** - Los dos últimos sistemas de recomendación irían destinados a los clientes que realizan los pedidos semanalmente para que obtengan sugerencias en función de sus gustos:

En algunos de estos sistemas se creará una segunda versión que, o bien añade información nueva, o bien se aproxima desde un enfoque diferente.



# Objetivos de los Sistemas de Recomendación

**Objetivo general de los sistemas de recomendación**

Mejorar la eficiencia operativa y la experiencia de los usuarios en la cocina comunitaria mediante la automatización de la planificación de los menús y la personalización de las sugerencias de los platos.

**Actividades**



1.   Diseñar y desarrollar un sistema de recomendación que identifique los platos más populares por familia, facilitando la creación de menús variados y atractivos para los usuarios.

2.   Implementar un sistema de recomendación que genere listas aleatorias de platos por familias, introduciendo un elemento de sorpresa y variabilidad en la oferta de menús semanales.

3.   Crear un sistema de recomendación que sugiera platos a los usuarios en función de sus preferencias y en las similitudes en los gustos con otros usuarios, mejorando la personalización de las recomendaciones.

4.   Desarrollar un sistema de recomendación basado en los ingredientes y que sugiera platos similares en función de las preferencias de los usuarios.

5.   Integrar los sistemas de recomendación en la operación diaria de la cocina comunitaria garantizando su correcto funcionamiento y su contribución positiva a las actividades de la asociación.

---

---

# Desarrollo de los sistemas de recomendación.

Desarrollaremos los sistemas de recomendación en un proceso compuesto en 3 fases:

- Extracción de los datos desde la base de datos en AWS
- Preprocesamiento y preparación de los datos. Generación de los dataframes para los modelos.
- Creación de los sistemas de recomendación.

## **Primera fase:** Extracción de los datos desde la base de datos en AWS

Nuestra base de datos se encuentra alojada en una instancia de RDS en AWS. Vamos a configurar la conexión y a importar algunas librerías para operar con las tablas que extraigamos.

In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector

In [ ]:
# Configuración de la conexión
host = 
user = 
password = 
database = 
port = 

try:
    # Establecer la conexión
    conexion = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=port
    )

    if conexion.is_connected():
        print("Conexión exitosa a la base de datos")

except mysql.connector.Error as error:
    print("Error al conectarse a la base de datos: {}".format(error))


Conexión exitosa a la base de datos


In [ ]:
# Ver las tablas que contiene
# Crear un cursor para ejecutar consultas
cursor = conexion.cursor()

# Consulta para obtener la lista de tablas
consulta_tablas = "SHOW TABLES"

# Ejecutar la consulta
cursor.execute(consulta_tablas)

# Obtener los resultados
tablas = cursor.fetchall()

# Mostrar las tablas
print("Tablas disponibles:")
for tabla in tablas:
    print(tabla[0])


Tablas disponibles:
allergens
families
menu_pick_up_times
menu_recipes
menu_type_families
menu_types
menus
migrations
options
order_item_adaptions
order_items
orders
pick_up_points
pick_up_times
recipe_allergens
recipe_options
recipes
slots
subscription_allergens
subscription_options
subscriptions
users


## **Segunda fase:** Generamos los dataframes que necesitaremos y aplicamos algunas labores de preprocesamiento de los datos



Del total de tablas que disponemos en la base de datos, únicamente son relevantes para nuestros modelos los siguientes:

- families - Contiene las familias a las que pertenecen los platos
- orders - Contiene los pedidos que realizan los usuarios
- order_items - Contiene el id de los platos que hay en cada pedido realizado por los usuarios
- recipies - Nos da el nombre del plato obtenido del id del dataframe anterior

En algunas tablas realizaremos algunas modificaciones y correcciones de los datos para adecuarlos a los posteriores procesos de modelado.

La información que disponemos viene en un formato con los nombres de las familias de los platos y el nombre de dichos platos en Euskera y en Castellano al mismo tiempo. Corregiremos este problema. Además corregiremos también algunos problemas de codificación.

In [ ]:
# Vamos a crear una lista con las tablas transformadas a dataframes de Pandas

import pandas as pd

# Convertir los resultados a una lista de nombres de tablas
nombres_tablas = [tabla[0] for tabla in tablas]

# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Crear DataFrames para cada tabla
for nombre_tabla in nombres_tablas:
    consulta_datos = f"SELECT * FROM {nombre_tabla}"
    cursor.execute(consulta_datos)
    datos = cursor.fetchall()
    column_names = [i[0] for i in cursor.description]
    df = pd.DataFrame(datos, columns=column_names)
    dataframes[nombre_tabla] = df


### Primera tabla: "families"

In [ ]:
families = pd.read_sql("SELECT * FROM families", conexion,  index_col= 'id')

# De esta tabla sólo necesitaremos algunas columnas
families = families["name"]
families

<ipython-input-94-4971d782a9a8>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  families = pd.read_sql("SELECT * FROM families", conexion,  index_col= 'id')


id
1                                 Lekaleak || Legumbres
2                                 Barazkiak || Verduras
8                            Osagarriak || Complementos
10    Pastak, Arrozak eta Entsaladak || Pastas, Arro...
12                 Kremak eta Saldak || Cremas y Caldos
15                                    Arraia || Pescado
16                                     Haragia || Carne
17                                          Ogia || Pan
18        Esneki eta Marmeladak || Lacteos y mermeladas
19                Etxeko azkenburuak || Postres caseros
20                              Sorginkeria || Brujeria
21                    Laguntzaileak || AcompaÃ±amientos
22                                  Helatuak || Helados
23                               Tortillak || Tortillas
24                                    Saltsak || Salsas
25    Haragi gordinak eta hestebeteak || Carnes crud...
26                                     Pastak || Pastas
27                                   Arrozak 

Vemos que los nombres de las familias de los platos contienen los dos idiomas: en euskera y en castellano. Para facilitar su comprensión eliminamos la parte en euskera.

In [ ]:
# Vamos a crear una función para obtener la parte en castellano
# Lo que haremos es eliminar toda la parte a la izquierda del símbolo ("||")
def obtener_parte_castellano(nombre):
    partes = nombre.split("||")
    return partes[-1] if len(partes) > 1 else nombre


In [ ]:
# Aplicamos la función a la serie "families"
families = families.apply(obtener_parte_castellano)

families

id
1                        Legumbres
2                         Verduras
8                     Complementos
10     Pastas, Arroces y ensaladas
12                 Cremas y Caldos
15                         Pescado
16                           Carne
17                             Pan
18            Lacteos y mermeladas
19                 Postres caseros
20                        Brujeria
21                AcompaÃ±amientos
22                         Helados
23                       Tortillas
24                          Salsas
25      Carnes crudas y embutidos 
26                          Pastas
27                         Arroces
28                       Ensaladas
29                         Bebidas
30    Errigora balioak. Produktuak
Name: name, dtype: object

Vemos que además hay un problema de codificación en los datos (se puede observar en el id 21 "AcompaÃ±amientos"). Son problemas de codificación UTF-8 en donde algunos caracteres como la ñ en castellano o la tilde han sido mal codificados. Corregimos este problema:

In [ ]:
# Función para corregir problemas de codificación
def corregir_codificacion(texto):
    try:
        texto_corregido = texto.encode('latin1').decode('utf-8')
        return texto_corregido
    except:
        return texto

In [ ]:
# Aplicamos la función a la serie "families"
families = families.apply(corregir_codificacion)

families

id
1                        Legumbres
2                         Verduras
8                     Complementos
10     Pastas, Arroces y ensaladas
12                 Cremas y Caldos
15                         Pescado
16                           Carne
17                             Pan
18            Lacteos y mermeladas
19                 Postres caseros
20                        Brujeria
21                 Acompañamientos
22                         Helados
23                       Tortillas
24                          Salsas
25      Carnes crudas y embutidos 
26                          Pastas
27                         Arroces
28                       Ensaladas
29                         Bebidas
30    Errigora balioak. Produktuak
Name: name, dtype: object

Por último, vamos a ponerlo todo en minúsculas

In [ ]:
families = families.str.lower()

families

id
1                        legumbres
2                         verduras
8                     complementos
10     pastas, arroces y ensaladas
12                 cremas y caldos
15                         pescado
16                           carne
17                             pan
18            lacteos y mermeladas
19                 postres caseros
20                        brujeria
21                 acompañamientos
22                         helados
23                       tortillas
24                          salsas
25      carnes crudas y embutidos 
26                          pastas
27                         arroces
28                       ensaladas
29                         bebidas
30    errigora balioak. produktuak
Name: name, dtype: object

###  Segunda tabla: "orders"

In [ ]:
orders = pd.read_sql("SELECT * FROM orders", conexion, index_col= 'id')

# Sólo necesitaremos algunas columnas
orders = orders["user_id"]

orders.tail()

<ipython-input-100-43d8351317bc>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql("SELECT * FROM orders", conexion, index_col= 'id')


id
2573    105
2574    126
2575    128
2576    166
2577    171
Name: user_id, dtype: int64

### Tercera tabla: "order_items"

In [ ]:
order_items = pd.read_sql("SELECT * FROM order_items", conexion, index_col= 'id')

# Sólo necesitaremos algunas columnas
order_items = order_items[["order_id","recipe_id"]]

order_items.tail()

<ipython-input-101-f1ad134f1e48>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_items = pd.read_sql("SELECT * FROM order_items", conexion, index_col= 'id')


,order_id,recipe_id
id,,
14682,2574,87
14683,2575,82
14684,2575,93
14685,2575,75
14686,2575,89


### Cuarta tabla: "recipes"

In [ ]:
recipes = pd.read_sql("SELECT * FROM recipes", conexion, index_col= 'id')

# Sólo necesitaremos algunas columnas
recipes = recipes[['name', 'family_id']]

recipes.head(100)

<ipython-input-102-1db1074900a7>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recipes = pd.read_sql("SELECT * FROM recipes", conexion, index_col= 'id')


,name,family_id
id,,
20,Arrai pastela || pastel de pescado,15
56,Oilasko fideua || fideuÃ¡ de pollo,16
64,Entsalada mistoa || Ensalada mixta,28
65,Entsaladilla errusiara || Ensaladilla rusa,28
66,Fruta-mazedonia || macedonia de frutas,28
...,...,...
162,Errioxako patatak || Patatas a la riojana,2
163,Amuarrain basatia ardo zurian || Trucha salvaj...,15
164,Oilasko pad thay || Pad thay de pollo,16


Vemos que los nombres de las columnas contienen el nombre del plato en euskera y en castellano. Para facilitar su comprensión eliminamos la parte en euskera.

In [ ]:
# Aplicamos la función que creamos anteriormente
recipes['name'] = recipes['name'].apply(obtener_parte_castellano)

recipes

,name,family_id
id,,
20,pastel de pescado,15
56,fideuÃ¡ de pollo,16
64,Ensalada mixta,28
65,Ensaladilla rusa,28
66,macedonia de frutas,28
...,...,...
208,Osobuco de ternera con setas,16
209,Bonito c/ piperrada,15
210,Ensalada griega,28


Corregimos también los problemas de codificación

In [ ]:
recipes['name'] = recipes['name'].apply(corregir_codificacion)

recipes

,name,family_id
id,,
20,pastel de pescado,15
56,fideuá de pollo,16
64,Ensalada mixta,28
65,Ensaladilla rusa,28
66,macedonia de frutas,28
...,...,...
208,Osobuco de ternera con setas,16
209,Bonito c/ piperrada,15
210,Ensalada griega,28


Lo ponemos todo a minúsculas

In [ ]:
recipes['name'] = recipes['name'].str.lower()

recipes

,name,family_id
id,,
20,pastel de pescado,15
56,fideuá de pollo,16
64,ensalada mixta,28
65,ensaladilla rusa,28
66,macedonia de frutas,28
...,...,...
208,osobuco de ternera con setas,16
209,bonito c/ piperrada,15
210,ensalada griega,28


## **Tercera fase:** Creación de los sistemas de recomendación

### Paso inicial - Vamos a crear un dataframe que nos muestre el id del usuario y la cantidad de platos que ha consumido.

In [ ]:
# Paso 1: Combinar la información
combined_df = pd.merge(orders, order_items, left_index=True, right_on='order_id')
combined_df = pd.merge(combined_df, recipes, left_on='recipe_id', right_index=True)

# Paso 2: Agrupar y contar
user_plate_counts = combined_df.groupby(['user_id', 'name']).size().reset_index(name='count')

# Paso 3: Pivote
pivot_table = user_plate_counts.pivot(index='user_id', columns='name', values='count').fillna(0)

pivot_table

name,"""suquet"" de bacalao","""trinxat de la cerdanya""",albondigas,alcachofas,"alcachofas, habas y jamón",alubia blanca,alubia pinta,alubias rojas de obanos,arroz a la bolognesa,arroz a la napolitana,...,txitxarro al horno,verdel en salsa verde,verduras al horno c/ romesco,verduras gratinadas con beixamel,vichysoyse,yogur de litro,bio.k zukuak 750ml,calabacín gratinado relleno de setas,pimientos del piquillo enteros,txapata
user_id,,,,,,,,,,,,,,,,,,,,,
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,3.0,0.0,12.0,0.0,3.0,0.0,0.0,0.0,4.0,2.0,...,0.0,1.0,1.0,0.0,2.0,29.0,17.0,1.0,0.0,0.0
47,2.0,1.0,15.0,0.0,8.0,2.0,0.0,7.0,3.0,2.0,...,1.0,1.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0
48,0.0,0.0,3.0,0.0,1.0,7.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
51,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Con este dataframe podemos ver el id del usuario y la cantidad de veces que ha consumido un plato. Este dataframe es fundamental para la elaboración de los posteriores sistemas de recomendación.

---

### Creación de los sistemas de recomendación

Crearemos 4 sistemas de recomendación que abordan las problemáticas identificadas y que supondrá al equipo de cocina de Sukalde una ayuda adicional que se traducirá en una carga de trabajo y tiempo mucho menor, y al mismo tiempo, un aumento en la satisfacción de los usuarios:

- El primer sistema de recomendación se centrará en ofrecer los platos más populares por familia. A través de esta herramienta el equipo de cocina podrá identificar rápidamente las preferencias de los usuarios y simplificar la creación de los menús. Supondrá una planificación más eficiente y una reducción del tiempo dedicado a esta tarea.

- En segundo lugar, el siguiente sistema proporcionará listas aleatorias de platos por familias, permitiendo una mayor variabilidad en la oferta de menús. Se creará una segunda versión de este sistema de recomendación en donde se introducirá un nuevo plato que todavía no se haya cocinado en la asociación. De esta forma se introduce un elemento sorpresa a la hora de crear los menús y elegir los platos por parte del usuario. Se mantiene el interés y la satisfacción de los usuarios a lo largo del tiempo.

- A continuación, se creará el tercer sistema de recomendación que estará enfocado en sugerir platos a los usuarios en función de sus preferencias y los gustos similares de otros usuarios. Esto permitirá a los clientes recibir recomendaciones personalizadas. Se consigue así aumentar su satisfacción a encontrar platos que todavía no ha probado y que se ajusten a sus preferencias individuales. Se creará una segunda versión en donde se utilizará la biblioteca Surprise con los algoritmos de SVD y KNN.

- Por último, crearemos un sistema de recomendación a través de los ingredientes que contienen los platos, de esta forma podemos sugerir a los usuarios platos similares en función de los ingredientes de los platos que más les gustan.

---
Por tanto, en función del uso al que van destinados los sistemas de recomendación, podemos dividirlos en dos grupos:

A) **Sistemas de recomendación para el equipo de cocina** - Los dos primeros sistemas de recomendación irán destinados al equipo de cocina de la asociación, de manera que les ayude a la creación de nuevos menús semana a semana.

B) **Sistemas de recomendación para los usuarios** - Los dos últimos sistemas de recomendación irían destinados a los usuarios que realizan los pedidos semanalmente para que obtengan sugerencias en función de sus gustos:

---



---
### A) **Sistemas de recomendación equipo de cocina**

Los siguientes sistemas de recomendación son para uso interno del equipo de cocina para la elaboración de los menús semanales.

#### **Sistema de recomendación 1**
Recomendación de platos por familia basado en la popularidad

In [ ]:
# Vamos a calcular la popularidad de cada plato basado en las veces que se ha consumido

popularidad = pivot_table.sum()
popularidad.sort_values(ascending=False)

name
 menestra de verduras              581.0
 lentejas                          513.0
 ensaladilla rusa                  477.0
 piquillos rellenos de pescado     469.0
 carne guisada                     310.0
                                   ...  
 menestra de verduras ecologica      1.0
 pasta bio martinelli                1.0
 dentrífico en polvo artesano        1.0
pimientos del piquillo enteros       1.0
 esparragos                          1.0
Length: 126, dtype: float64

In [ ]:
# Cambiamos el nombre de la columna
popularidad.name = 'Popularidad'

# Creamos la función para que nos haga recomendaciones por familia de platos
def recomendacion_por_familia(family_id, n=5):
    recetas_familia = recipes[recipes['family_id'] == family_id]
    top_platos_familia = recetas_familia.merge(popularidad, left_on='name', right_index=True)
    top_platos_familia = top_platos_familia.sort_values(by='Popularidad', ascending=False).head(n)
    return top_platos_familia['name']

# Ejemplo de recomendación para la familia de legumbres (tenemos que ver la lista de más arriba ("families") para asignar la recomendación al tipo que deseemos)
recom_legumbre = recomendacion_por_familia(1, n=5) # El valor 1 corresponde a legumbres
print("5 platos recomendados para legumbres:")
recom_legumbre


5 platos recomendados para legumbres:


id
82                    lentejas
81                   garbanzos
85     alubias rojas de obanos
83               alubia blanca
84                alubia pinta
Name: name, dtype: object

Vemos otro ejemplo: pescados.

In [ ]:
# Tenemos que ver que en la lista de arriba "pescados" tiene asignado el id = 15.

print("5 platos recomendados para pescados:")
recom_pescados = recomendacion_por_familia(15, n=5)
recom_pescados

5 platos recomendados para pescados:


id
96      piquillos rellenos de pescado
20                  pastel de pescado
73                   lassagna de atún
104            merluza en salsa verde
125        arroz integral de txipiron
Name: name, dtype: object

Por lo tanto, nuestra función `recomendacion_por_familia` ofrecerá al equipo de cocina sugerencias en cuanto a la popularidad de los platos por familia para elaborar sus menús semanales.

---

#### **Sistema de recomendación 2**
Recomendación de platos por familia basado en aleatoreidad

Generaremos dos diferentes versiones del sistema de platos aleatorio

**Versión 1:**

In [ ]:
# Ofrecer nuevos platos en cada familia
def nuevos_platos(family_id, n=5):
    platos_familia = recipes[recipes['family_id'] == family_id]
    nuevos_platos = platos_familia.sample(n)  # Muestra aleatoria de platos dentro de la familia
    return nuevos_platos['name']

# Ejemplo de oferta de nuevos platos para la familia de legumbres (suponiendo que el ID de legumbres sea 1)
legumbres = nuevos_platos(1, n=5) # Asignamos el valor 1 correspondiente a legumbres.
print("Nuevos platos de legumbres:")
print(legumbres)


Nuevos platos de legumbres:
id
82                    lentejas
83               alubia blanca
84                alubia pinta
85     alubias rojas de obanos
81                   garbanzos
Name: name, dtype: object


Siempre que ejecutemos nuevamente el código anterior generaremos una nueva lista de platos por familia.

Por lo tanto, nuestra función `nuevos_platos` ofrecerá una lista aleatoria de platos por familia cada semana para que el equipo de cocina pueda incorporar cierta rotación/aleatoreidad en la elaboración de sus menús.

---

**Versión 2:**

La versión 1 es un sistema de recomendación muy sencillo que únicamente introduce aleatoreidad por familia de platos. En búsqueda de aumentar la satisfacción y mantener el interés de los usuarios, se ha propuesto que el sistema de recomendación además de ofrecer una lista de platos aleatorios por familia, añada además un plato nuevo que nunca se haya preparado dentro de un grupo de familias.

Para ello, se van a introducir nuevos platos en algunas de las familias, concretamente:

- Legumbres
- Verduras
- Pastas, arroces y ensaladas
- Pescados
- Carnes
- Lácteos y mermeladas
- Postres caseros

In [ ]:
# Agregamos el archivo que nos ofrecerá los nuevos platos por familia a nuestro sistema de recomendación

platos_nuevos = pd.read_excel("platos_nuevos.xlsx")
platos_nuevos

,id_recipies,name,family_id
0,300,Lentejas estofadas,1
1,301,Garbanzos con espinacas,1
2,302,Fabada asturiana,1
3,303,Hummus,1
4,304,Guiso de alubias,1
...,...,...,...
75,375,Natillas,19
76,376,Crumble de frutas,19
77,377,Flan casero,19
78,378,Pastel de tres leches,19


In [ ]:
# Creamos la nueva versión del sistema de recomendación

# Importamos la biblioteca de random para elegir un plato aleatorio dentro del archivo que acabamos de cargar

import random

def nuevos_platos_v2(family_id, n=5):
    platos_familia = recipes[recipes['family_id'] == family_id]

    # Obtener una lista de los platos nuevos para la familia
    nuevos_platos_familia = platos_nuevos[platos_nuevos['family_id'] == family_id]['name'].tolist()

    # Elegir un plato nuevo aleatorio si hay disponibles
    plato_nuevo = random.choice(nuevos_platos_familia) if nuevos_platos_familia else None

    # Elegir n-1 platos aleatorios de la familia actual
    platos_aleatorios = platos_familia.sample(n - 1)['name'].tolist()

    # Combinar los platos aleatorios con el plato nuevo (si existe)
    if plato_nuevo:
        platos_aleatorios.append(plato_nuevo)

    return platos_aleatorios


In [ ]:
# Ejemplo para platos de carne (le corresponde el id = 16). Vamos a seleccionar una lista aleatoria de 7 platos.

carnes = nuevos_platos_v2(16,n=7)

print ("Nuevos platos de carne:")
carnes


Nuevos platos de carne:


[' sopa de pollo',
 ' potaje de lentejas',
 ' fideuá con pollo y hongos',
 ' potaje de alubias de tolosa',
 ' piquillos de cocido',
 ' muslo de pollo al txilindron',
 'Pollo al horno']

Como vemos en esta nueva versión del código ```nuevos_platos_v2``` generamos una lista de n-1 platos aleatorios dentro de la familia y añadimos además uno nuevo que nunca se ha preparado. Se ha dejado la primera letra mayúscula en los nuevos platos para que se pueda diferenciar de los anteriores.



---

### **B) Sistemas de recomendación usuarios**

Los siguientes sistemas de recomendación serán utilizados por los usuarios de la asociación para obtener recomendaciones de platos nuevos.

#### **Sistema de recomendación 3**
Recomendación basado en usuarios similares


**Versión 1**

Para el siguiente sistema de recomendación vamos a usar métodos de **filtrado colaborativo**. Para ello:

- Calcularemos las similitudes entre usuarios
- Seleccionaremos usuarios similares
- Generaremos recomendaciones

Antes de efectuar el sistema hemos de elegir la métrica adecuada para la similitud. Disponemos de varias opciones: Jaccard, coseno, correlación de Pearson y la distancia Euclidiana.

Debido a la naturaleza de los datos y al objetivo que perseguimos, nos decantaremos por utilizar la similitud del coseno.

En cuanto a la naturaleza de los datos, la elección se basa en que ésta métrica considera la presencia de cada uno de los platos del usuario y la cantidad de los platos que ha consumido. La similitud de Jaccard no considera las veces que se consumió un plato por lo que no parece la más adecuada. Por otro lado, la magnitud de las cantidades nos puede generar problemas con la distancia euclidiana.

En cuanto al objetivo que perseguimos, la correlación de Pearson no es la más apropiada, mide la similitud entre usuarios en función de cómo sus preferencias varían juntas y nosotros no deseamos identificar patrones de comportamiento similares entre usuarios, sino generar recomendaciones entre usuarios similares para que prueben platos que aún no han probado.



##### - Calculamos similitud

In [ ]:
# Importamos la función para calcular la similitud del coseno
from sklearn.metrics.pairwise import cosine_similarity

# Calculamos la matriz de similitud entre usuarios
matriz_similitud = cosine_similarity(pivot_table)

matriz_similitud

array([[1.        , 0.36849966, 0.24664109, ..., 0.        , 0.94491118,
        0.        ],
       [0.36849966, 1.        , 0.48653443, ..., 0.12710477, 0.34595301,
        0.04707824],
       [0.24664109, 0.48653443, 1.        , ..., 0.11042687, 0.32627549,
        0.26180967],
       ...,
       [0.        , 0.12710477, 0.11042687, ..., 1.        , 0.        ,
        0.23570226],
       [0.94491118, 0.34595301, 0.32627549, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.04707824, 0.26180967, ..., 0.23570226, 0.        ,
        1.        ]])

##### - Seleccionamos usuarios similares

In [ ]:
# Creamos la función

def usuarios_similares(user_id, threshold=0.5): # Determinamos el umbral de similitud como mínimo a un 50%

    user_id_idx = pivot_table.index.get_loc(user_id) # Es importante señalar que esta línea de código nos permite ajustarnos al índice real del dataframe
                                                     # pivot_table. Este dataframe no tiene un índice autoincremental y para desarrollar el código de una manera
                                                     # más eficiente convertimos esos índices del dataframe en índices por su ubicación real en dicho dataframe.

    similitud_usuario = matriz_similitud[user_id_idx]

    # Creamos la lista de usuarios similares con un bucle que nos vaya recorriendo la variable anterior. Tenemos que determinar el umbral de similitud
    # para que sólo nos devuelva por cada usuario aquellos con un valor superior al umbral.
    usuarios_similares = [(idx, sim) for idx, sim in enumerate(similitud_usuario) if idx != user_id_idx and sim >= threshold]

    # Una vez que hemos hallado los usuarios_similares, es preciso convertir los índices de similaridad nuevamente a user_id reales para
    # asignar correctamente su valor con el índice que ocupa en el dataframe de pivot_table.

    usuarios_similares = [(pivot_table.index[idx], sim) for idx, sim in usuarios_similares]

    usuarios_similares.sort(key=lambda x: x[1], reverse=True)
    return usuarios_similares


Probamos la función:

In [ ]:
# Elegimos al azar un usuario para ver aquellos usuarios similares y su similitud

user_id = 47
usuario = usuarios_similares(user_id)

# Sacamos los 10 usuarios con mejor similitud

top_10_usuarios = usuario[:10]

print(f"Top 10 - Usuarios similares al usuario {user_id}")
print("-"*32)
for usuario, similitud in top_10_usuarios:
    print(f"Usuario {usuario} - Similitud: {(similitud*100).round(2)}%")

Top 10 - Usuarios similares al usuario 47
--------------------------------
Usuario 71 - Similitud: 79.99%
Usuario 67 - Similitud: 79.19%
Usuario 110 - Similitud: 78.64%
Usuario 100 - Similitud: 77.24%
Usuario 146 - Similitud: 76.87%
Usuario 118 - Similitud: 76.8%
Usuario 56 - Similitud: 76.25%
Usuario 119 - Similitud: 75.68%
Usuario 89 - Similitud: 75.68%
Usuario 77 - Similitud: 74.73%


Vemos que funciona correctamente, creamos las recomendaciones

##### - Generación de recomendaciones

In [ ]:
def obtener_recomendaciones(user_id, threshold=0.5):

    # Para esta nueva función hemos de obtener la lista de usuarios similares que habíamos sacado anteriormente
    usuarios_similares_lista = usuarios_similares(user_id, threshold)

    # Ahora la lista de platos que el usuario de interés ha probado (lo necesitaremos para el bucle que queremos generar)
    platos_probados = pivot_table.loc[user_id]

    # Obtenemos la lista de platos disponibles que nos servirá también para el bucle
    platos_disponibles = pivot_table.columns

    # Calculamos la puntuación de recomendación para cada plato no probado y lo metemos en una lista
    recomendaciones = []
    for plato in platos_disponibles:
        if platos_probados[plato] == 0:  # De esta manera verificamos que el usuario no ha probado el plato
            puntuacion = 0
            for usuario_similar, similitud in usuarios_similares_lista:
                puntuacion += similitud * pivot_table.loc[usuario_similar, plato]
            recomendaciones.append((plato, puntuacion)) # Vamos llenando la lista

    # Ahora ordenamos la lista de recomendaciones en función de la puntuación de recomendación
    recomendaciones.sort(key=lambda x: x[1], reverse=True)

    # Obtenemos los platos recomendados
    return recomendaciones

In [ ]:
# Ejemplo de uso con el mismo usuario del paso anterior:

user_id = 47
recomendaciones = obtener_recomendaciones(user_id, threshold=0.5)

# Sacamos las 10 mejores recomendaciones

top_10_recomendaciones = recomendaciones[:10]

print(f"Top 10 - Platos recomendados al usuario {user_id}")
print("-"*32)
for plato, puntuacion in top_10_recomendaciones:
    print(f"{plato} \t - Con una puntuación del: {(puntuacion).round(2)}%")

Top 10 - Platos recomendados al usuario 47
--------------------------------
 ensalada de patatas 	 - Con una puntuación del: 54.13%
 ensalada de arroz 	 - Con una puntuación del: 40.88%
 yogur de litro 	 - Con una puntuación del: 36.25%
 caldo de carne 	 - Con una puntuación del: 22.15%
 bacalao c/ piperrada 	 - Con una puntuación del: 19.36%
 octavo de queso 	 - Con una puntuación del: 18.47%
txapata 	 - Con una puntuación del: 15.94%
 pan de cereales 	 - Con una puntuación del: 14.93%
 macedonia de frutas 	 - Con una puntuación del: 14.35%
 pan de maiz y espelta 	 - Con una puntuación del: 12.75%


A través de la función `"obtener_recomendaciones"` creada podemos ver los platos recomendados al usuario y la puntuación que obtiene dentro de la similitud con los usuarios que son afines a sus gustos.

---


Versión 2 - Utilizamos la biblioteca Surprise

El uso de esta biblioteca es adecuado para la creación de este sistema de recomendación, puesto que está basado en filtrado colaborativo (finalidad para la que está diseñada esta biblioteca). Además, nos permite implementar diferentes algoritmos y evaluar su desempeño.

Para poder utilizar la biblioteca Surprise necesitamos el rating de los usuarios a los platos que han consumido. Como no disponemos de esta información vamos a efectuar una transformación basada en la frecuencia relativa de consumo de un plato como la valoración que ofrece un usuario.

In [ ]:
!pip install scikit-surprise

In [ ]:
from surprise import Dataset
from surprise import Reader

data = []
for usuario, row in pivot_table.iterrows():
    total_platos_consumidos = row.sum()  # Total de platos que ha consumido el usuario
    for plato, frecuencia in row.items():
        # Calculamos la calificación ficticia basada en la frecuencia de consumo de un plato
        # La calificación será el cociente entre las veces que ha consumido ese plato y el total de platos consumidos
        if total_platos_consumidos == 0:
            rating = 0  # Para evitar el problema de que nos haga la división por cero si el usuario no ha consumido ningún plato
        else:
            rating = frecuencia / total_platos_consumidos
        data.append((usuario, plato, rating))

min_rating = 0
max_rating = 1

# Creamos un objeto Reader para indicar el rango de calificaciones
reader = Reader(rating_scale=(min_rating, max_rating))

# Convierte la lista anterior a un DataFrame de pandas
df = pd.DataFrame(data, columns=['user', 'item', 'rating'])

# Crea un conjunto de datos Surprise desde el DataFrame
dataset = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

In [ ]:
# Establecemos los conjuntos de prueba y test
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)


- Algoritmo SVD

In [ ]:
from surprise import SVD
from surprise import accuracy

# Creamos una instancia del algoritmo SVD
algo = SVD()

# Entrenamos el modelo en el conjunto de entrenamiento
algo.fit(trainset)

# Realizamos las predicciones en el conjunto de prueba
predictions = algo.test(testset)

# Calculamos el RMSE (Error Cuadrático Medio) y el MAE (Error Absoluto Medio)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


RMSE: 0.0477
MAE:  0.0297


Valores cercanos a cero de estas dos métricas nos indican que nuestro modelo tiene buen rendimiento y es preciso en las predicciones. Por lo tanto, parece que tiene buen desempeño.

In [ ]:
from collections import defaultdict

# Creamos un diccionario para almacenar las recomendaciones para cada usuario
top_n = defaultdict(list)

# Obtén las predicciones para todos los usuarios en el conjunto de prueba
for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))

# Ordenamos las predicciones por usuario. Establecemos 5 como el total de recomendaciones.
n = 5
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:n]

# Sacamos las recomendaciones para un usuario específico
user_id = 47
print(f'Recomendaciones para el usuario {user_id}:')
for item_id, est in top_n[user_id]:
    print(f'Plato: {item_id}, Estimación de calificación: {est * 100:.2f}%')

Recomendaciones para el usuario 47:
Plato:  caldo de pollo, Estimación de calificación: 14.07%
Plato:  arroz integral c/ curry amarillo, Estimación de calificación: 12.08%
Plato:  ensaladilla rusa, Estimación de calificación: 9.45%
Plato:  berenjenas rellenas de pollo al curry, Estimación de calificación: 7.99%
Plato:  pastel de pescado, Estimación de calificación: 6.88%


Estas serían las recomendaciones propuestas al utilizar el algoritmo SVD para el usuario 47.

- Algoritmo KNN

In [ ]:
from surprise import KNNBasic

algo2 = KNNBasic()
algo2.fit(trainset)

predictions2 = algo2.test(testset)
rmse = accuracy.rmse(predictions2)
mae = accuracy.mae(predictions2)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0184
MAE:  0.0088


Los resultados son todavía mejores que los del algoritmo anterior por lo que el modelo con el algoritmo KNN funciona mejor que con SVD.

In [ ]:
top_n2 = defaultdict(list)

for uid, iid, true_r, est, _ in predictions2:
    top_n2[uid].append((iid, est))

n = 5  # Número de recomendaciones a generar

for uid, user_ratings in top_n2.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n2[uid] = user_ratings[:n]

# Imprime las recomendaciones para un usuario específico
user_id = 47
print(f'Recomendaciones para el usuario {user_id}:')
for item_id, est in top_n2[user_id]:
    print(f'Plato: {item_id}  \t Calificación: {est * 100:.2f}%')


Recomendaciones para el usuario 47:
Plato:  ensaladilla rusa  	 Calificación: 4.06%
Plato:  pastel de pescado  	 Calificación: 2.75%
Plato:  crema de calabacín  	 Calificación: 1.27%
Plato:  ensalada mixta  	 Calificación: 1.22%
Plato:  tallarín de espinaca con crema de idiazabal y aceituna negra   	 Calificación: 0.91%


Estas serían las recomendaciones del algoritmo KNN para el usuario 47.

---

#### **Sistema de recomendación 4**
Recomendación basado en los ingredientes de los platos

Vamos a generar ahora un sistema de recomendación que le proponga al usuario platos que se asemejen a sus platos favoritos en función de los ingredientes que contienen.

Para ello debemos importar un archivo con las fichas técnicas de los platos en donde aparecen sus ingredientes.

In [ ]:
platos = pd.read_excel("platos_final.xlsx")

# Vemos el contenido del archivo

platos

,Unnamed: 0,TIPO,PLATO,INGREDIENTES,Q,Ud,€/Ud,%IVA,/PAX,Raciones,Ingrediente/pers,Id_recipes,family_id,Id_proveedores
0,0,Verduras,Patatas a la riojana,Patata,800.0,g,0.9984,0.1,0.2496,4,200.0,162,2,Patatas gaztamina
1,1,Verduras,Patatas a la riojana,Chorizo,200.0,g,2.9900,0.1,0.7475,4,50.0,162,2,Carniceria peruene
2,2,Verduras,Patatas a la riojana,Pimiento rojo,100.0,g,2.8800,0.1,0.7200,4,25.0,162,2,Barazki mattiak
3,3,Verduras,Patatas a la riojana,Pimiento verde,80.0,g,4.0000,0.1,1.0000,4,20.0,162,2,Barazki mattiak
4,4,Verduras,Patatas a la riojana,Cebolla,200.0,g,0.7212,0.1,0.1803,4,50.0,162,2,Barazki mattiak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,743,NaN,Galletas de chocolate,Azúcar,800.0,g,NaN,0.1,NaN,4,200.0,211,19,NaN
744,744,NaN,Galletas de chocolate,Mantequilla,400.0,g,NaN,0.1,NaN,4,100.0,211,19,NaN
745,745,NaN,Lassagna de bolognesa y setas,Placas lassagna (50ud),10.0,Ud,NaN,0.1,NaN,4,2.0,212,16,NaN
746,746,NaN,Lassagna de bolognesa y setas,Boloñesa,1000.0,g,NaN,0.1,NaN,4,250.0,212,16,NaN


In [ ]:
# Nos quedamos sólo con las columnas relevantes para nuestro estudio

platos = platos[["PLATO","INGREDIENTES","Id_recipes"]]
platos

,PLATO,INGREDIENTES,Id_recipes
0,Patatas a la riojana,Patata,162
1,Patatas a la riojana,Chorizo,162
2,Patatas a la riojana,Pimiento rojo,162
3,Patatas a la riojana,Pimiento verde,162
4,Patatas a la riojana,Cebolla,162
...,...,...,...
743,Galletas de chocolate,Azúcar,211
744,Galletas de chocolate,Mantequilla,211
745,Lassagna de bolognesa y setas,Placas lassagna (50ud),212
746,Lassagna de bolognesa y setas,Boloñesa,212


Debemos hacer algunas transformaciones en nuestro dataframe anterior "pivot_table" para sustituir el nombre del plato que tienen en las columnas por el id que le corresponde.

Esta operación es necesaria porque nos facilita cruzar ese dataframe de manera más eficiente con nuestro dataframe recién creado.

In [ ]:
# Para hacerlo crearemos un diccionario con los platos y sus índices y lo mapearemos en las columnas del siguiente dataframe
dic_indices_platos = dict(zip(recipes['name'], recipes.index))

# Cambiamos los nombres de columna utilizando el diccionario
pivot_table.columns = pivot_table.columns.map(dic_indices_platos)


In [ ]:
pivot_table

name,103,132,107,188,174,83,84,85,92,93,...,136,177,183,151,181,113,171,200,191,115
user_id,,,,,,,,,,,,,,,,,,,,,
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,3.0,0.0,12.0,0.0,3.0,0.0,0.0,0.0,4.0,2.0,...,0.0,1.0,1.0,0.0,2.0,29.0,17.0,1.0,0.0,0.0
47,2.0,1.0,15.0,0.0,8.0,2.0,0.0,7.0,3.0,2.0,...,1.0,1.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0
48,0.0,0.0,3.0,0.0,1.0,7.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
51,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ahora ya tenemos los dos dataframes para poder generar nuestro último sistema de recomendación.

En éste último sistema de recomendación utilizaremos la métrica jaccard_score que previamente no escogimos debido a la naturaleza de la información y el objetivo que perseguíamos.

En este caso tiene sentido utilizar dicha métrica puesto que tratamos de calcular **similitud entre conjuntos de ingredientes**. Si dos platos son dos conjuntos diferentes, la intersección de ambos platos representa los ingredientes que comparten ambos platos y si un ingrediente aparece en un plato tendrá un valor de 1 y si no aparece tendrá un valor de 0.

Debido a estas circunstancias, la métrica de similitud a utilizar que más se ajusta a nuestro objetivo ahora es la métrica de Jaccard. Se adapta bien a la naturaleza de los datos y permite calcular la similitud de manera más eficiente.

In [ ]:
from sklearn.metrics import jaccard_score

def recomendacion_por_ingredientes(user_id, n=5):

    # Identificamos los platos favoritos del usuario
    platos_favoritos = pivot_table.loc[user_id]
    platos_favoritos = platos_favoritos[platos_favoritos > 0].index.tolist()

    # Obtenemos los ingredientes de los platos favoritos
    ingredientes_favoritos = platos[platos['Id_recipes'].isin(platos_favoritos)][['PLATO', 'INGREDIENTES']]

    # Creamos un diccionario para almacenar la puntuación total de similitud por plato
    puntuaciones_platos = {}

    for index, row in platos.iterrows():
        plato_actual = row['PLATO']
        ingredientes_plato_actual = row['INGREDIENTES']

        # Calculamos la similitud de ingredientes con los platos favoritos usando Jaccard
        similitud = 0
        total_ingredientes = len(ingredientes_favoritos)

        for ingrediente_favorito in ingredientes_favoritos['INGREDIENTES']:
            similitud += int(ingrediente_favorito in ingredientes_plato_actual)

        similitud = similitud / total_ingredientes

        # Debemos asignar ahora la similitud a la puntuación total del plato
        puntuaciones_platos[plato_actual] = similitud

    # Vamos a ordenar los platos por puntuación de similitud en orden descendente
    platos_ordenados = sorted(puntuaciones_platos.items(), key=lambda x: x[1], reverse=True)

    # Por último, para evitar que el código nos devuelva platos ya consumidos por el usuario debemos añadir las siguientes líneas
    platos_consumidos = pivot_table.loc[user_id]
    platos_consumidos = platos_consumidos[platos_consumidos > 0].index.tolist()

    platos_recomendados = [(plato, puntuacion) for plato, puntuacion in platos_ordenados if plato not in platos_consumidos]

    recomendaciones = platos_recomendados[:n]

    return recomendaciones

In [ ]:

user_id = 47  # Seguimos con el mismo ejemplo

recomendaciones = recomendacion_por_ingredientes(user_id, n=5)
print(f"5 platos recomendados para el usuario {user_id} basados en los ingredientes de sus platos favoritos:")
print("-"*32)
for plato, puntuacion in recomendaciones:
    print(f"Plato: {plato}, Puntuación de Similitud: {round(puntuacion*100, 2)}%")

5 platos recomendados para el usuario 47 basados en los ingredientes de sus platos favoritos:
--------------------------------
Plato: Arroz integral c/ curry amarillo, Puntuación de Similitud: 7.53%
Plato: Fideuá con pollo y hongos:, Puntuación de Similitud: 7.53%
Plato: Guisantes, alcachofas y jamon, Puntuación de Similitud: 7.53%
Plato: Corvina en salsa verde, Puntuación de Similitud: 7.53%
Plato: Trinxat de la cerdanya, Puntuación de Similitud: 7.53%


Para obtener el valor de la similitud dividimos los ingredientes que se encuentran en el plato por el total de ingredientes. Al tener una lista tan grande de ingredientes y un conjunto de platos bastante diverso obtenemos esta similitud que parece algo baja. En otras palabras, para la gran variedad de ingredientes de que disponemos, no disponemos en contraparte de una muestra suficiente de platos que permita al algoritmo encontrar platos similares.

A medida que el equipo de cocina de Sukalde vaya introduciendo nuevos platos, el algoritmo encontrará cada vez más platos similares y se reducirá esa diferencia entre la cantidad de ingredientes de que se dispone y las veces que un ingrediente aparece en un plato.





# Evaluación

El proceso de creación de los sistemas de recomendación ha constado de diferentes etapas bien establecidas que han culminado en la obtención de los sistemas deseados con resultados acordes a nuestras expectativas.

Para una mayor comprensión del trabajo realizado se ha considerado relevante establecer dos grupos diferenciados dentro de los sistemas de recomendación en función del uso y respuesta de la necesidad: 2 para el equipo de cocina y 2 para los usuarios.

En algunos de estos sistemas se ha realizado una segunda versión, en la que se añadía nueva información (Sistema de Recomendación 2 - Versión 2) o en la que se probaban nuevos algoritmos (Sistema de Recomendación 3 - Versión 2).

Para garantizar que los resultados sean útiles y coherentes en la construcción de los sistemas de recomendación ha sido necesario aplicar una serie de transformaciones y correcciones previas. Dentro de la base de datos que tenemos alojada en AWS necesitábamos algunas de las tablas que contenían la información necesaria con los usuarios y los pedidos. Hemos aplicado las correcciones necesarias y nos hemos quedado con la información precisa que necesitábamos para la construcción de los sistemas. Posteriormente ha sido necesario utilizar otros documentos adicionales con las fichas técnicas de los platos y otro con sugerencias de nuevos platos para los usuarios.



---



A la hora de comenzar con nuestro trabajo para crear los sistemas de recomendación se estuvo analizando la información de la que disponíamos para ver de qué manera podríamos usar la biblioteca Surprise para realizar los sistemas.

La biblioteca Surprise está diseñada para la creación y evaluación de sistemas de recomendación, sobre todo para el filtrado corborativo. De manera, que en lugar de componer nuestro algoritmo de manera manual desde cero, podríamos simplificar la tarea y usar diferentes algoritmos para evaluar su comportamiento. Una vez establecidos los 4 sistemas de recomendación necesarios para nuestro proyecto, nos dimos cuenta que donde mejor funcionaría Surprise era a la hora de elaborar el sistema de recomendación número 3, basado en filtrado colaborativo.

Sin embargo, los datos que teníamos para establecer las recomendaciones no se adaptaban a la forma en que esta biblioteca trabaja. Por ejemplo, a la hora de diseñar este "Sistema de recomendación 3", teníamos el problema de que necesitábamos **las calificaciones de los usuarios** para poder implementar los diferentes algoritmos. Como no disponíamos de esta información, adaptamos nuestro dataset para ofrecer calificaciones ficticias.

No hemos encontrado trabajos parecidos en donde se utilizara Surprise del mismo modo en que nosotros lo queríamos implementar. Así que probamos a generar una calificación en base a esta información de la que disponíamos.

Una aproximación a esta solución era convertir las frecuencias de consumo en calificaciones ficticias, asignando una calificación más alta a los platos que se consumieron con mayor frecuencia. De tal forma que aunque no representan una calificación real, podíamos tratarlos como calificaciones en el sistema de recomendación. Se consideró utilizar un sistema binario de calificación (1 si había consumido el plato y 0 si no lo había consumido) pero había una gran pérdida de información.

Al final, se realizaron dos modelos: uno con SVD y otro con KNN en la versión 2 de nuestro "Sistema de recomendación 3". Analizaremos los resultados en la siguiente sección.

Por último, cabe mencionar que en principio habíamos contemplado un único sistema de recomendación para los usuarios (el "Sistema de recomendación 3" con sus dos versiones, basado en el filtrado colaborativo). Sin embargo, siguiendo algunos de los principales trabajos realizados en el área (Yum_me - Yang et al., 2017 y Trang Tran et al., 2018) en donde se generan sistemas para la recomendación de recetas y en donde se analizan los ingredientes de recetas junto a la preferencia de los usuarios, decidimos agregar un nuevo sistema debido a que disponíamos de la información necesaria (las fichas técnicas de los platos con los ingredientes). Lo cierto es que en comparación con los otros estudios nuestros datos tenían bastantes limitaciones puesto que no disponíamos de un perfil del usuario tan desarrollado como en esos modelos, ni los componentes nutricionales de las recetas. De tal forma, que tuvimos que ajustar y preparar nuestro algoritmo desde cero, ajustándonos a nuestra capacidad de implementación, porque no tendríamos los parámetros necesarios para ejecutar el algoritmo de forma eficiente.

# Resultados

## - Sistema de recomendación 1:

Este sistema está basado en la popularidad de familias. Es un algoritmo sencillo que funciona de manera efectiva para proporcionar los platos populares dentro de cada categoría de familia de platos. Responde a la demanda del equipo de cocina en cuanto a la planificación de los menús, reducción del tiempo de creación de los menús y seguimiento de los platos más populares por los miembros de la asociación.

A medida que la base de datos se vaya actualizando semana a semana, los resultados de estas tablas se irán modificando y los miembros del equipo de cocina pueden ir viendo la evolución de los platos más populares dentro de cada familia para ir adaptando los menús en función de sus necesidades. Por ejemplo, si tenemos un excedente de algún ingrediente en una semana, podemos considerar cómo incorporarlo en alguno de estos platos.

## - Sistema de recomendación 2:

Para este sistema de recomendación hemos generado dos versiones.

Versión 1 - Es un sistema basado en la aleatoreidad por familias. Nuevamente la primera versión de este sistema es un algoritmo muy sencillo que únicamente muestra de manera aleatoria los platos por familia. Responde también a las necesidades del equipo de cocina a la hora de considerar una mayor variabilidad en la oferta de los menús.

Versión 2 - Introduce nuevos platos no cocinados previamente en la lista anterior. Responde a la necesidad del equipo de cocina de Sukalde porque añade un elemento sorpresa en la selección de platos por los usuarios y puede conducir a mantener el interés y la satisfacción de los mismos al ofrecer variedad en la selección de platos.

##- Sistema de recomendación 3:

En este sistema de recomendación hemos generado también dos versiones y en la segunda versión hemos probado dos algoritmos.

Versión 1 -
Estamos bastante satisfechos con los resultados que nos ofrece esta primera versión del sistema de recomendación por similitud entre los usuarios. Es una versión que hemos diseñado casi al completo, funciona de manera efectiva y mantiene coherencia en los resultados al basarse en la matriz de similitudes entre usuarios.

A través de la matriz de similitud, cuyo empleo del coseno ha sido debidamente justificado, se detectan los usuarios similares y en función de ellos se establecen las recomendaciones, sugiriendo platos que el usuario nunca ha consumido.

Los resultados que ofrecen las diferentes recomendaciones nos parecen precisos y con fundamentación matemática válida.

Versión 2 -
La segunda versión de nuestro sistema de recomendación nos genera más dudas al respecto. A pesar de que hemos conseguido adaptar nuestros datos para implementar la biblioteca Surprise y utilizar diferentes algoritmos, no consideramos la métrica que hemos utilizado como válida para aceptar los resultados.

Obtenemos buenos resultados de RMSE y MAE en ambos algoritmos.

```
KNN                 |       SVD                |
RMSE: 0.0184        |       RMSE: 0.0477       |
MAE:  0.0088        |       MAE:  0.0297       |
```


Tienen valores bajos, con lo que nuestros modelos son precisos en sus predicciones y el modelo realiza recomendaciones que son muy similares a las preferencias reales de los usuarios. Sin embargo, consideramos que obtenemos buenos resultados bajo una premisa equivocada.

La aproximación que hemos utilizado como calificación de los usuarios condiciona estos resultados. Que un usuario haya elegido un plato el doble de veces que otro, no debería suponer que uno tenga una calificación 2 veces superior. Pudiera ser que una semana ese plato no podía ofertarse, con lo que las veces que ha elegido ese plato al final ha sido menor.

Por esta razón, aun a pesar de las buenas métricas obtenidas para los algoritmos de SVD y KNN en la versión 2 de nuestro tercer sistema de recomendación, recomendamos usar la primera versión. Sus resultados son más precisos y respaldados con mejor fundamentación matemática.

## - Sistema de recomendación 4:

El último sistema de recomendación estaba basado en recomendación por ingredientes.

Hemos justificado el uso de la métrica Jaccard en este último sistema, ya que tratamos de calcular similitud entre conjuntos de ingredientes y para esto, es la métrica adecuada.

Como podemos ver, los resultados obtenidos en el usuario de prueba tienen un valor de similitud no demasiado elevado:

```
5 platos recomendados para el usuario 47 basados en los ingredientes de sus platos favoritos:
--------------------------------
Plato: Arroz integral c/ curry amarillo, Puntuación de Similitud: 7.53%
Plato: Fideuá con pollo y hongos:, Puntuación de Similitud: 7.53%
Plato: Guisantes, alcachofas y jamon, Puntuación de Similitud: 7.53%
Plato: Corvina en salsa verde, Puntuación de Similitud: 7.53%
Plato: Trinxat de la cerdanya, Puntuación de Similitud: 7.53%
```

Por un lado podemos observar que todos los platos tienen la misma similitud y puede ser indicativo de que podemos estar cometiendo algún error, sin embargo podemos probar solicitando un número mayor de platos y comprobar que funciona bien:

```
Plato: Osobuco de ternera con setas, Puntuación de Similitud: 5.73%
Plato: Salsa bolognesa, Puntuación de Similitud: 5.02%
Plato: Bizcocho de chocolate y zanahor, Puntuación de Similitud: 3.23%
Plato: Ensalada alubia y bacalao, Puntuación de Similitud: 3.23%
Plato: Ensalada de garbanzo, Puntuación de Similitud: 3.23%
```
Y por otro lado, podemos ver el puntaje de similitud del plato en función de los ingredientes que contienen sus platos favoritos. Como se justificaba al final del sistema de recomendación 4, para la gran variedad de ingredientes de que disponemos, no disponemos en contraparte de una muestra suficiente de platos que permita al algoritmo encontrar platos similares, de tal forma que obtenemos valores con un puntaje porcentual algo bajo.

El único problema es que para obtener puntajes elevados es preciso además, que los platos sean muy similares a aquellos favoritos de nuestros usuarios. De esa manera, el algoritmo encuentra un número total mayor de ingredientes en un plato nuevo del conjunto de sus platos favoritos. Dicho de otra manera, si de 10 de mis ingredientes favoritos encuentro 7 en un mismo plato, tendré un puntaje elevado, pero seguramente me indicaría que es un plato muy similar.

En definitiva, nuevamente recomendamos como sistema de recomendación para usuarios la primera versión del "Sistema de recomendación 3" puesto que en este caso no disponemos de un conjunto de datos con una información suficiente que nos pueda ofrecer unas recomendaciones significativas.


# Conclusiones y trabajos futuros

La división que hemos realizado de los sistemas de recomendación en dos grupos refleja la versatilidad de la solución para abordar tanto las necesidades internas de la asociación (derivadas con la planificación y producción de los menús), como las necesidades de los usuarios finales que buscan opciones de menús que se adapten mejor a sus gustos personales.

Una vez que se comiencen a usar los sistemas de recomendación los resultados esperados a raíz de este trabajo son:

- Mejora de la eficiencia operativa de Sukalde en términos de la simplicidad en la planificación de los menús y la reducción del tiempo dedicado a dichas tareas.

- Mayor satisfacción de los usuarios al obtener sugerencias personalizadas y una oferta de menús más variada y ajustada a sus gustos.

---

Las cocinas comunitarias, aunque no son necesariamente proyectos recientes, han surgido en los últimos años en áreas urbanas como parte de movimientos más amplios relacionados con la sostenibilidad, la alimentación local, la seguridad alimentaria y la promoción de la comunidad.

Para ofrecer servicios de mayor calidad y tener un impacto más significativo en sus comunidades, la implementación de prácticas y procesos más sólidos contribuye directamente a la mejora de sus capacidades. Los sistemas de recomendación aportan un potencial de mejora en la eficiencia operativa, la satisfacción del usuario e incluso en la gestión de los recursos. Pueden conducir a una mayor sostenibilidad financiera y a una reducción de costos operativos por un lado, y mejorar la experiencia culinaria y la satisfacción de los usuarios por otro. En cierta medida, pueden fortalecer la colaboración y la interacción social al permitir que los usuarios participen en la elección de recetas y alimentos.

Más allá del contexto de las cocinas comunitarias, los sistemas de recomendación tienen también un gran potencial en el mundo de la restauración en general. Pueden ofrecer una serie de beneficios tanto a los restaurantes como a sus clientes.

Trabajos futuros:

La cuestión principal que nos surge a la mente una vez completado el trabajo de los sistemas de recomendación pasan por una recogida con datos que muestren más información, y un mejor ajuste y mejor desarrollo de estos algoritmos. Para ello, la información disponible de los platos y las preferencias o restricciones de los usuarios deben ser también más elaboradas. En este sentido, se podrían desarrollar algoritmos que tuvieran en cuenta las preferencias dietéticas de los usuarios o sus intolerancias alimentarias. Pero para ello se deberían integrar los datos nutricionales de los alimentos y de los ingredientes de los platos y contemplar los alérgenos o intolerancias de los usuarios.

En definitiva, los sistemas de recomendación diseñados contribuirán directamente a la mejora de la eficiencia operativa de la cocina de Sukalde y a la satisfacción de sus miembros, aunque se identifican espacios de mejora que requieren un mayor trabajo e investigación para la optimización y generalización de estos sistemas, para adaptarlos en mejor medida a las necesidades específicas de cada cocina.



